In [ ]:
import pygame # It is a module which is used to make games and other multimedia applications.
import random # Defines a series of functions for generating random integers and figures.
from pygame import mixer # Used to add music to our game.


pygame.init() # After importing we need to initialize pygame. 


Screen= Width, Height = 300, 500 # These are the Dimensions of our game board. 
Window = pygame.display.set_mode(Screen, pygame.NOFRAME) # We need to initialize a window where our game will display.
                                         # (This is the Syntax to display a window)
# We use pygame.NOFRAME to remove the the caption line from thr top of game board.

# Used to insert background at the Play Window.
bgimg = pygame.image.load("BackGround2.png") 
bgimg = pygame.transform.scale(bgimg, (Width, Height-120)).convert_alpha() 
# Convert.alpha is used to blit any background 
# Without affecting the game (Game speed, FPS).


# This code is used to make grids to our game board. It will show how many Rows and Cols are made in Game board. 
# There is no Direct way to obtain the Grids in python, So we initialize Small squares. 
# And with the help of loops we marked Vertical and horizontal lines to show Grid Structures.
CellSize = 20
Rows = (Height-120) // CellSize
Cols = Width // CellSize
#print(Rows, Cols) -> This will print 19 rows and 15 Columns.


# This is used to introduce FPS to our game.
clock = pygame.time.Clock()
FPS = 24

# This is used to add music to the game background of the game.
mixer.music.load('BGsound.mp3')
mixer.music.play(-1) # This is used to play the repeatedly again and again.

# Introducing Colours to our screen:
# These are called RGB colour combinations.(All colours are formed by these combinations)
Black = (21, 24, 29)
Blue = (31, 25, 76)
Red = (252, 91, 122)
White = (255, 255, 255)


# These are the tetris block images downloaded from browser and we used them in our game.
img1 = pygame.image.load('Assets/1.png')
img2 = pygame.image.load('Assets/2.png')
img3 = pygame.image.load('Assets/3.png')
img4 = pygame.image.load('Assets/4.png')


# In this we store our images in Dictionary format to access them later in our code.
Assets = {
    1 : img1,
    2 : img2,
    3 : img3,
    4 : img4
}


# Used Pre-loaded and system fonts to represent score system and levels.
font = pygame.font.Font('Fonts/Alternity-8w7J.ttf', 50)
font2 = pygame.font.SysFont('cursive', 25)


# Here We initialize a class to represent Figures in Tetris.
class Tetramino:
    # Matrix Figures
    # [0   1   2   3]
    # [4   5   6   7]
    # [8   9   10  11]
    # [12  13  14  15]

    Figures = {
        'I' : [[1, 5, 9, 13], [4, 5, 6, 7]],
        'Z' : [[4, 5, 9, 10], [2, 6, 5, 9]],
        'S' : [[6, 7, 9, 10], [1, 5, 6, 10]],
        'L' : [[1, 2, 5, 9], [0, 4, 5, 6], [1, 5, 9, 8], [4, 5, 6, 10]],
        'J' : [[1, 2, 6, 10], [5, 6, 7, 9], [2, 6, 10, 11], [3, 5, 6, 7]],
        'T' : [[1, 4, 5, 6], [1, 4, 5, 9], [4, 5, 6, 9], [1, 5, 6, 9]],
        'O' : [[1, 2, 5, 6]]
    }
    Types = ['I', 'Z', 'S', 'L', 'J', 'T', 'O'] # Store these figures in a list 

    def __init__(self, x, y): # The __init__ function is called every time an object is created from a class.
        self.x = x # Self function is used to access variables that belongs to the class.       
        self.y = y
        self.type = random.choice(self.Types) # It will take random alphabet of shapes from the list.
        self.shape = self.Figures[self.type] # It will take the figure from the 2-D list associated to random alphabet.
        self.color = random.randint(1, 4) # Displaying our loaded images and give random colors to Matrix figures.
        self.rotation = 0 # Rotation will be initialize as zero. It will take the first element from 2D list. 

    def image(self):
        return self.shape[self.rotation]# It will take the 1st element from the random 2D list.

    def rotate(self):
        self.rotation = (self.rotation + 1) % len(self.shape) # It will rotate the shapes.
    # For eg.- Randomly 'L' shape is taken, Now firstly it represents the figure at 0 index then rotate it by + 1 and keep 
    # doing until we stop pressing the up key and '%' sign is used to get all the shapes of a paticular element.

# Explanation:
#print(t.type)
#print(t.shape)
#print(t.color)
#print(t.image())
# Z -> t.type will take Z as random figure.
#[[4, 5, 9, 10], [2, 6, 5, 9]] -> t.shape gives the matrix format of figure associated with Z.
#4 -> t.color represents the random 4th colour from the list of colors.
#[4, 5, 9, 10] -> t.image() represents the first figure of Z and if we put rotation + 1< It will rotate the same figure. 
    


class Tetris: # We initialize a class where we can store various functions inside it to make our code more clearer.
    def __init__(self, rows, cols): # This is used to create our board so that we can perform various tasks on it.
        self.rows = rows # Initializing rows
        self.cols = cols # Initializing cols
        self.score = 0 # Initializing score
        self.level = 1 # Initializing level
        self.board = [[0 for j in range(cols)] for i in range(rows)] # We use list comprehension to form a 2D list of Game board
        self.next = None # Initializing next as none, This will help us to represent the upcoming figure in tetris.
        self.gameover = False # Initializing Gameover as false. 
        self.new_figure() # This will help to generate new figures.



    def new_figure(self):
        if not self.next:
            self.next = Tetramino(5, 0) # This will generate new figure.
        self.figure = self.next # The new figure generated stored to self.figure .
        self.next = Tetramino(5, 0) # # This will store the new upcoming figure.

    def intersects(self): # To Enable collsion between sides and bottom of game board, Defined function intersects.
        intersection = False # This means currently figure is not colliding with any other figure or sides.
        for i in range(4):  # This for loop checks between the 4 x 4 Matrix figures.  
            for j in range(4):
                if i * 4 + j in self.figure.image(): # This checks for each block in the tetris 4 x 4 matrix.
                    if i + self.figure.y > self.rows - 1 or \
                    j + self.figure.x > self.cols - 1 or \
                    j + self.figure.x < 0 or \
                    self.board[i + self.figure.y][j + self.figure.x] > 0: # Check Collison with other shapes on game board.
                        # The above lines checks collision between right side, Left side and at the bottom of game board.
                        intersection = True # This enables the collision conditions.
        return intersection # Returns the function.

    def remove_line(self): # This is used to remove completed line from the bottom of game board.
        rerun = False # Initializing rerun as false when condition becomes true, code will execute.                  
        for y in range(self.rows-1, 0, -1): # Start from bottom to top of game board.
            is_full = True # Considering last to be full with tetris shapes. 
            for x in range(0, self.cols):
                if self.board[y][x] == 0: # This condition checks whether the last line is completed or not.
                    is_full = False # If not, condition becomes false and line will remove and shifts to top.
            if is_full: # If condition becomes true it will perform the follwing tasks.
                del self.board[y] # This will deletes the last line 
                self.board.insert(0, [0 for i in range(self.cols)]) # Adds empty line at the top of game board.
                self.score += 1 # Increases score by one.
                if self.score % 5 == 0: # Checks if score becomes 5 or not.
                    self.level += 1 # If score becomes 5, it will increase the level by 1.
                rerun = True # If conditions meet this will run the code.

        if rerun:
            self.remove_line() # Calling function.

    def freeze(self):
        for i in range(4):
            for j in range(4):
                if i * 4 + j in self.figure.image():
                    self.board[i + self.figure.y][j + self.figure.x] = self.figure.color
        self.remove_line() # Calls remove line function here.
        self.new_figure() # Genersting new figures.
        if self.intersects(): # If shapes are colliding each other at the top then game overs.
            self.gameover = True # This will over the game when conditions meet. 

    def go_space(self):
        while not self.intersects():
            self.figure.y += 1
        self.figure.y -= 1
        self.freeze()

    def go_down(self):
        self.figure.y += 1
        if self.intersects():
            self.figure.y -= 1
            self.freeze()

    def go_side(self, dx):
        self.figure.x += dx
        if self.intersects():
            self.figure.x -= dx

    def rotate(self):
        rotation = self.figure.rotation
        self.figure.rotate()
        if self.intersects():
            self.figure.rotation = rotation


counter = 0
move_down = False
can_move = True


tetris = Tetris(Rows, Cols)
        

# From here we introduce our while loop and performs various tasks: Caliing our functions used in classes and so on.
running = True # This Syntax is used to continue running of a screen.
while running: # what this while loop does, It will continuously appears our game window on screen. 
    Window.fill(Black) # This will fill Black colour to the game window
    Window.blit(bgimg,(0, 0)) # Used to fill background to the game.

    # This Counter is define to give speed to our falling shapes.
    counter += 1
    if counter >= 10000:
        counter = 0

    if can_move:
        if counter % (FPS // (tetris.level * 2)) == 0 or move_down:
            if not tetris.gameover:
                tetris.go_down()

                
    for event in pygame.event.get():# This 'for loop' and 'event function' is used to disappear the game window if we click on
                                 # cross or Quit icon.
        if event.type == pygame.QUIT:
            running = False  # This will close the window

        if event.type == pygame.KEYDOWN:
            if can_move and not tetris.gameover:
                
                if event.key == pygame.K_LEFT: # On Pressing left arrow key, Shape will move leftwards.
                    tetris.go_side(-1)

                if event.key == pygame.K_RIGHT: # On Pressing right arrow key, Shape will move rightwards.
                    tetris.go_side(1)

                if event.key == pygame.K_UP: # On Pressing Up arrow key, Shape will rotates.
                    tetris.rotate()

                if event.key == pygame.K_DOWN: # On Pressing down arrow key, Shape will move downwards.
                    move_down = True

                if event.key == pygame.K_SPACE: # On Pressing Space bar key, Shape will fall directly on bottom of game board.
                    tetris.go_space()

            if event.key == pygame.K_r: # On Pressing r, Game will restart.
                mixer.music.play() # After restart, Used to play the song again.
                tetris.__init__(Rows, Cols)

            if event.key == pygame.K_p: # On pressing p, Game will pause.
                mixer.music.pause() # This helps to pause the Background song when game pauses.
                can_move = not can_move
                if can_move:
                    mixer.music.play() # This helps to again play the song when game continues.

            if event.key == pygame.K_q or event.key == pygame.K_ESCAPE: # On pressing q or esc, game will quit
                running = False

        if event.type == pygame.KEYUP:
            if event.key == pygame.K_DOWN:
                move_down = False

    #tetris.draw_grid() # We comment this line to remove the grid from the tetris game board.

# Displaying Board 
    for x in range(Rows):
        for y in range(Cols):
            if tetris.board[x][y] > 0:
                val = tetris.board[x][y]
                img = Assets[val]
                Window.blit(img, (y*CellSize, x*CellSize))
                pygame.draw.rect(Window, White, (y*CellSize, x*CellSize, CellSize, CellSize), 1) # This is used to give white
                # Outline to our shapes to look more attractive.

                
# This section of code is used to display figures in game board. 
    if tetris.figure:
        for i in range(4):# This represents the number of particular shape in matrix lists.
            for j in range(4):
                if i * 4 + j in tetris.figure.image():# This will represent each element of list in 2D list.
                                                   #(ex- 'Z' : [[4, 5, 9, 10]])
                    img = Assets[tetris.figure.color] # This is used to give random color to shape 
                    x = CellSize * (tetris.figure.x + j)
                    y = CellSize * (tetris.figure.y + i) # This will represent Random shapes of tetris and fix them to CellSize.
                    Window.blit(img, (x, y)) # This represent the tetris shapes on another screen,
                    pygame.draw.rect(Window, White, (x, y, CellSize, CellSize), 1)

# This section of code represents the Game over screen.
    if tetris.gameover:
        mixer.music.stop() # Used to stop the music when game ends/overs.
        rect = pygame.Rect((50, 140, Width-100, Height-350)) # These are the dimensions of gameover screen.
        pygame.draw.rect(Window, Black, rect)  
        pygame.draw.rect(Window, Red, rect, 2)

        
        # These are the fonts (particular) we want to represent on the gameover screen.
        over = font2.render('Game Over!!', True,  White)
        msg1 = font2.render('Press R to restart', True, White)
        msg2 = font2.render('Press Q to quit', True, White)
        
        # These will represent the above lines on the gameover screen.
        Window.blit(over, (rect.centerx-over.get_width()/2, rect.y + 20))
        Window.blit(msg1, (rect.centerx-msg1.get_width()/2, rect.y + 80))
        Window.blit(msg2, (rect.centerx-msg2.get_width()/2, rect.y + 110))


    pygame.draw.rect(Window, Blue , (0, Height-120, Width, 120))# This code will divide our game window into 2 separate halves 
                                                                # i.e play window and information window.

    if tetris.next: # Calls tetris.next function which will  represent the new upcoming shape.
        for i in range(4):
            for j in range(4):
                if i * 4 + j in tetris.next.image():
                    img = Assets[tetris.next.color] # This provides color to upcoming image.
                    x = CellSize * (tetris.next.x + j - 4)
                    y = Height - 100 + CellSize * (tetris.next.y + i) # These are the dimensions where our upcoming shape shows.
                    Window.blit(img, (x, y)) # This represents the upcoming shape at the bottom of screen.

    scoreimg = font.render(f'{tetris.score}', True, White) # Creating score screen 
    levelimg = font2.render(f'Level : {tetris.level}', True, White) # Creating level screen 
    Window.blit(scoreimg, (250-scoreimg.get_width()//2, Height-110)) # This will represent the score on game board.
    Window.blit(levelimg, (250-levelimg.get_width()//2, Height-30)) # This will represent the level on the game board.

    pygame.draw.rect(Window, Black, (0, 0, Width, Height-120), 2) # This is used to make Border to two different halves.
    clock.tick(FPS) # Calls our fps function.
    pygame.display.update() # This 'update' function updates the screen(Game window) everytime we change anything in code.

pygame.quit() # It will end the game